In [1]:
!nvidia-smi

Thu Dec 31 06:42:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html


     |████████████████████████████████| 55.0MB 54kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 8.9MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588513 sha256=47279d8570de8b788fbe5cfeacce966cc1874975bf3eb25fdf328a02a9400756
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.1MB 9.1MB/s 
     |████████████████████████████████| 757kB 9.0MB/s 
     |████████████████████████████████| 890kB 18.7MB/s 
     |████████████████████████████████| 3.0MB 40.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=0b1e0b7f7bf7cff93433307893684fef7c1a8b91830ee4d79a57d104143c3caf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a5012

In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-coog0uy8
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-coog0uy8
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=9e8330b6d00a4b35a2d6ee102037c505f7c0c3c59a3f6e076fad4bd496620dd6
  Stored in directory: /tmp/pip-ephem-wheel-cache-8micxd0n/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
##GPU 사용 시
device = torch.device("cuda:0")

In [8]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [9]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [10]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [11]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [12]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [13]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [14]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [15]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [16]:
from google.colab import drive
isMount=!df |grep /content/drive
if len(isMount) < 1 : drive.mount('/content/drive')


Mounted at /content/drive


In [17]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# train

In [ ]:
## Setting parameters
max_len = 64
batch_size = 10
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
!wget https://raw.githubusercontent.com/BussNam/Funding-classify/main/WadizData.tsv


In [20]:
dataset_train = nlp.data.TSVDataset("WadizData.tsv", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("WadizData.tsv", field_indices=[1,2], num_discard_samples=1)

In [21]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [22]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)


In [23]:
t_total = len(test_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [24]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/Lec_Capture/_dataNLP/myBertModel"))


<All keys matched successfully>

In [29]:

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


epoch 1 batch id 1 loss 0.0022819957230240107 train acc 1.0

epoch 1 train acc 1.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 1.0


epoch 2 batch id 1 loss 0.0022112857550382614 train acc 1.0

epoch 2 train acc 1.0


...


epoch 8 test acc 1.0


epoch 9 batch id 1 loss 0.0012843505246564746 train acc 1.0

epoch 9 train acc 1.0



epoch 9 test acc 1.0


epoch 10 batch id 1 loss 0.0011502227280288935 train acc 1.0

epoch 10 train acc 1.0



epoch 10 test acc 1.0


In [30]:
# Model Save
torch.save(model.state_dict(), "/content/drive/MyDrive/newsModel")


In [31]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/newsModel"))


<All keys matched successfully>

In [33]:

batch_size=21
dataset_test2 = nlp.data.TSVDataset("WadizData.tsv", field_indices=[1,2], num_discard_samples=1)

data_test2 = BERTDataset(dataset_test2, 0, 1, tok, max_len, True, False)
test_dataloader2 = torch.utils.data.DataLoader(data_test2, batch_size=batch_size, num_workers=5)

t_total = len(test_dataloader2) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

test_acc=0
label=0
result=0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader2)):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)
      out = model(token_ids, valid_length, segment_ids)
      test_acc += calc_accuracy(out, label)
      lable1=label.cpu().numpy()
      result=torch.max(out, 1)[1].cpu().numpy()
      print('-----------------')
      print("test_acc : ",test_acc)
      print("label  : ", lable1)
      print('result : ', result )



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


-----------------
test_acc :  1.0
label  :  [1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
result :  [1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]



In [35]:
!cat WadizData.tsv

id	document	label
1	3만원대 노트북 파우치에 높이조절 거치대를 담았다! 미세높이조절 53도!	1
2	맥북2대 2.5시간 완충! 120W-PD+70W-올인원A 멀티 고속충전기	1
3	퍼퓸계의 미슐랭 3-STAR. 죽기 전에 발라봐야 할 바디로션이 돌아왔다	1
4	[ 손톱 호강/5천원 ] 큐티클 순삭은 기본이죠! 유향미르 네일 에센스	1
5	물리치료사와 작업치료사가 연구, 개발한 손목보호대	1
6	[누적2.5억] 말랑말랑 "핏넥'베개가 일체형'원바디'가 되어 돌아왔어요	1
7	[임상시험완료! 꿀잠각] 수면이 숙면으로 바뀌는 수면캡슐이 40만원대에?	1
8	[평점 4.9점 앵콜] 보들보들 끝판왕 | 후기가 증명한 인생팬티!	1
9	올 겨울 대세템ㅣ머플러 + 후드 = 한섬이 만든 79그램 경량 머플러	1
10	집에서 레이싱경기를?! 마리오카트와 슈퍼카로 즐기는 360도 스피드레이싱	1
11	IT인프라산업투자ㅣ삼성전자,SK하이닉스 납품ㅣ88억매출달성	0
12	종료 D-dayㅣ공유킥보드 Top3기업, 더스윙	0
13	한국영화 기대작 3편 포트폴리오투자	0
14	천만 재테크족을 위한 혁신적 대안을 만들어가는 '불리오'	0
15	10주년 맞이, 그린플러그드 2019!	0
16	영화 투자ㅣ'20년 개봉ㅣ전도연,정우성/설경구,이선균 주연	0
17	마켓컬리부터 이마트까지,아시아 식품강자 TBM!	0
18	창업하고 싶은 착한 프랜차이즈 '커피에반하다'	0
19	펀딩률 300% 돌파ㅣ한류투자ㅣBTS데뷔 2400일의 여정	0
20	40만 흥행 원작, 러빙빈센트 전	0
